In [3]:
import numpy as np
from mrjob.job import MRJob

class MatrixMultiplication(MRJob):

    def mapper_A(self, key, value):

        for k in range(self.B.shape[0]):
            yield (key, k), ('A', value)

    def mapper_B(self, key, value):
        # برای هر خانه در ماتریس B، یک tuple با شماره سطر، شماره ستون و مقدار ایجاد می‌کنیم
        for i in range(self.A.shape[1]):
            yield (i, key), ('B', value)

    def reducer(self, key, values):

        values = list(values)
        a_values = [val for val in values if val[0] == 'A']
        b_values = [val for val in values if val[0] == 'B']
        a_dict = {k: v for (k, v) in a_values}
        b_dict = {k: v for (k, v) in b_values}
        result = sum([a_dict.get((key[0], k), 0) * b_dict.get((k, key[1]), 0) for k in range(self.B.shape[0])])
        yield key, result

    def steps(self):
        return [
            self.mr(mapper=self.mapper_A),
            self.mr(mapper=self.mapper_B),
            self.mr(reducer=self.reducer)
        ]




